## Deep Learning Classifier for Detecting DeepFake Forgeries in Videos

The model implemented was adapted from MesoNet-4 architecture from this [paper](https://www.google.com/url?q=https%3A%2F%2Farxiv.org%2Fpdf%2F1809.00888.pdf) by Afchar et al.


#### Overview of MesoNet-4

MesoNet-4 is composed of 4 CNN layers (with MaxPooling after each layer) followed by a fully connected layer with one hidden layer which outputs the probability of an image being real or fake.  

#### Architecture

Below is a figure from the paper detailing the network architecture.

<center><img src="https://drive.google.com/uc?export=view&id=1My8UQ-l3FijeBRGTG_YhoaY4JnA3xpP8" width=400/></center>

#### Dataset
The dataset has been provided by the MesoNet authors. The colored face images are aligned, of size `256x256`. Training and testing datasets include both real and deepfaked images

In [3]:
# Check if GPU exists - Tesla 4
!nvidia-smi

Mon Aug 19 13:35:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
# set the seed to get reproducable results
from numpy.random import seed
seed(1)
#from tensorflow import set_random_seed
import tensorflow
tensorflow.random.set_seed(2)

In [19]:
import tensorflow as tf

In [20]:
import keras

In [21]:
from keras.models import Model

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# get keras layers
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
# we will use the Adam optimizer for this assignment
from keras.optimizers import Adam

In [25]:
# define the image width
IMAGE_WIDTH = 256
# define the image height
IMAGE_HEIGHT = 256
# define the image channels
IMAGE_CHANNELS = 3
# define the learning rate
LEARNING_RATE = 0.0001
# batch size for the ImageDataGenerator
BATCHSIZE = 30

In [26]:
class MesoNet4:
    def __init__(self):
        self.model = self.get_model()

    def get_model(self):
        x = Input(shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

        # first block
        layer1 = Conv2D(8, (3,3), padding='same', activation='relu')(x)
        layer1 = BatchNormalization()(layer1)
        layer1 = MaxPooling2D(pool_size=(2,2), padding='same')(layer1)

        # second block
        layer2 = Conv2D(8, (5,5), padding='same', activation='relu')(layer1)
        layer2 = BatchNormalization()(layer2)
        layer2 = MaxPooling2D(pool_size=(2,2), padding='same')(layer2)

        # thrid block
        layer3 = Conv2D(16, (5,5), padding='same', activation='relu')(layer2)
        layer3 = BatchNormalization()(layer3)
        layer3 = MaxPooling2D(pool_size=(2,2), padding='same')(layer3)

        # fourth block
        layer4 = Conv2D(16, (5,5), padding='same', activation='relu')(layer3)
        layer4 = BatchNormalization()(layer4)
        conv_output = MaxPooling2D(pool_size=(4,4), padding='same')(layer4)

        # conv_output is the output of the last maxpool
        # we need to fallen the output of the conv layers
        y = Flatten()(conv_output)
        # add dropout layer
        y = Dropout(0.5)(y)
        # ADD THE FULLY CONNECTED 16
        y = Dense(16)(y)
        # add another dropout layer
        y = Dropout(0.5)(y)

        # y is the output of Dropout
        # we add one dense layer with sigmoid
        y = Dense(1, activation = 'sigmoid')(y)
        # now we return the model
        return Model(inputs = x, outputs = y)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/data/train/",
                                                    target_size = (256, 256),
                                                    batch_size = BATCHSIZE,
                                                    class_mode = 'binary')


Found 3525 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory("/content/drive/MyDrive/data/test/",
                                                    target_size = (256, 256),
                                                    batch_size = BATCHSIZE,
                                                    class_mode = 'binary')


Found 880 images belonging to 2 classes.


In [ ]:
# check the data
train_generator.class_indices

{'fake': 0, 'real': 1}

In [ ]:
test_generator.class_indices

{'fake': 0, 'real': 1}

In [ ]:
## define an optimizer
opt = Adam(learning_rate=LEARNING_RATE)

In [ ]:
## Compile the model
NETWORK = MesoNet4()
NETWORK.model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['accuracy'])

## Train the model

In [ ]:
# use NETWORK.model to access the model
# loss: 0.0604 - accuracy: 0.9325
EPOCHS = 3
trainer = NETWORK.model.fit_generator(generator=train_generator,
                                      steps_per_epoch = len(train_generator),
                                      epochs = EPOCHS,
                                      verbose =1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
118/118 [==============================] - 12s 104ms/step - loss: 0.2563 - accuracy: 0.7101
Epoch 2/3
118/118 [==============================] - 13s 106ms/step - loss: 0.0830 - accuracy: 0.9058
Epoch 3/3
118/118 [==============================] - 12s 106ms/step - loss: 0.0604 - accuracy: 0.9325


In [ ]:
NETWORK.model.save("/content/drive/MyDrive/meso")
NETWORK.model.save_weights("/content/drive/MyDrive/meso_weights")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/meso/assets


## Test the model

In [ ]:
# use NETWORK.model to access the model
# loss and metrics(accuracy)
test_scores = NETWORK.model.evaluate_generator(generator=test_generator)
print("The accuracy of the trained model is: ", test_scores[1])

Instructions for updating:
Please use Model.evaluate, which supports generators.
The accuracy of the trained model is:  0.8409090638160706


The training accuracy is 0.9325, and the accuracy of the trained model on test data is 0.8409090638160706